In [6]:
from langchain_deepseek import ChatDeepSeek
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime

model = ChatDeepSeek(
    model="deepseek-chat",
    #temperature=0,
    #max_tokens=None,
    #timeout=None,
    max_retries=2,
    # api_key="...",
    # other params...
)

@dataclass
class UserContext:
    user_id: str

model

ChatDeepSeek(client=<openai.resources.chat.completions.completions.Completions object at 0x00000215683DBB30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000215683DB1D0>, root_client=<openai.OpenAI object at 0x0000021566CF6630>, root_async_client=<openai.AsyncOpenAI object at 0x00000215683DBFE0>, model_name='deepseek-chat', model_kwargs={}, max_retries=2, api_key=SecretStr('**********'), api_base='https://api.deepseek.com/v1')

In [14]:



USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

@dataclass
class UserContext:
    user_id: str

@tool
def get_account_info(runtime: ToolRuntime[UserContext]) -> str:
    """Get the current user's account information."""
    user_id = runtime.context.user_id

    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"

agent = create_agent(
    model=model,
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You are a financial assistant."
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my current balance?"}]},
    context=UserContext(user_id="user123")
)
result["messages"][-1].content


"I'll check your account information to get your current balance."

In [15]:
from pathlib import Path
@tool
def list_files_tool(directory_path: str = ".") -> str:
    """列出指定目录中的所有文件和子目录。
    
    Args:
        directory_path: 要列出的目录路径，默认为当前目录
    
    Returns:
        目录中的文件和子目录列表
    """
    try:
        path = Path(directory_path)
        if not path.exists():
            return f"错误：目录 '{directory_path}' 不存在"
        
        if not path.is_dir():
            return f"错误：'{directory_path}' 不是一个目录"
        
        items = []
        for item in sorted(path.iterdir()):
            item_type = "📁" if item.is_dir() else "📄"
            items.append(f"{item_type} {item.name}")
        
        if not items:
            return f"目录 '{directory_path}' 是空的"
        
        return f"目录 '{directory_path}' 的内容：\n" + "\n".join(items)
    except Exception as e:
        return f"列出目录 '{directory_path}' 时出错: {str(e)}"

tools = [list_files_tool, get_account_info]
agent = create_agent(
    model=model,
    tools=tools,
    context_schema=UserContext,
    system_prompt="You are a helpful assistant."
)

Message = {
    "messages": [
        {
            "role": "user", 
            "content": "列出当前目录，并且查询我的账单"
        }
    ]
}
result = agent.invoke(Message,
                      context = UserContext(user_id="user123"))
result["messages"][-1].content

'当前目录包含以下文件：\n- 📄 agent.demo_flow.ipynb\n- 📄 data_handling.py\n- 📄 ochestration_agent.py\n- 📄 test_file_tools.py\n\n您的账户信息如下：\n- **账户持有人**: Alice Johnson\n- **账户类型**: Premium（高级账户）\n- **账户余额**: $5000'

In [16]:
@tool
def read_file_tool(file_path: str) -> str:
    """读取文件内容。输入文件路径，返回文件的文本内容。
    
    Args:
        file_path: 要读取的文件路径（相对路径或绝对路径）
    
    Returns:
        文件内容的字符串，如果出错则返回错误信息
    """
    try:
        path = Path(file_path)
        if not path.exists():
            return f"错误：文件 '{file_path}' 不存在"
        
        if not path.is_file():
            return f"错误：'{file_path}' 不是一个文件"
        
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        return f"文件 '{file_path}' 的内容：\n{content}"
    except UnicodeDecodeError:
        return f"错误：无法读取文件 '{file_path}'，可能是二进制文件"
    except Exception as e:
        return f"读取文件 '{file_path}' 时出错: {str(e)}"
    
tools = [list_files_tool, read_file_tool]

agent = create_agent(
    model=model,
    tools=tools,
    context_schema=UserContext,
    system_prompt="You are a helpful assistant."
)

Message = {
    "messages": [
        {
            "role": "user", 
            "content": "读取文件source\test_file_tools.py的内容 "
        }
    ]
}
result = agent.invoke(Message, context = UserContext(user_id="user123"))
result['messages'][-1].content


'我已经成功读取了 `test_file_tools.py` 文件的内容。这是一个用于测试文件操作工具的独立脚本，包含以下功能：\n\n1. **写入文件** - 创建一个测试文件并写入内容\n2. **读取文件** - 读取已创建的文件内容\n3. **列出目录** - 显示当前目录的文件列表\n4. **修改文件** - 替换文件中的特定内容\n5. **清理文件** - 删除测试文件\n\n该脚本从 `source.ochestration_agent` 模块导入了四个文件操作工具函数，并提供了一个完整的测试流程来验证这些工具的功能。'

In [19]:
import pandas as pd
import numpy as np
from typing import List, Optional, Dict, Any
from sklearn.tree import DecisionTreeClassifier
from langchain.tools import tool

# ============== 1. Binning helper =====================
@tool
def bin_single_feature_tool(
    series: List[float],
    y: Optional[List[float]] = None,
    method: str = "quantile",   # 'quantile' | 'width' | 'tree'
    n_bins: int = 10,
    min_leaf_frac: float = 0.05,
) -> Dict[str, Any]:
    """
    Bin a single numeric feature according to the specified method.

    Parameters
    ----------
    series : pd.Series
        Raw feature values.
    y : pd.Series, optional
        Target labels (only needed when method='tree').
    method : str
        'quantile' (equal frequency), 'width' (equal width),
        or 'tree' (decision-tree-based optimal binning).
    n_bins : int
        Number of bins or max leaf nodes (for tree).
    min_leaf_frac : float
        Minimum fraction of samples per leaf for tree binning.

    Returns
    -------
    pd.Series
        Series of bin labels (string), same index as input series.
        Missing values are grouped into 'MISSING'.
    """
    # convert to numeric, coerce invalid to NaN
    s = pd.to_numeric(series, errors="coerce")
    bins = pd.Series(index=s.index, dtype="object")

    # handle missing values as a separate bin
    missing_mask = s.isna()

    # ================== quantile / width ==================
    if method in ("quantile", "width"):
        valid_mask = ~missing_mask
        if valid_mask.sum() == 0:
            # all values are missing
            bins[missing_mask] = "MISSING"
            return bins

        s_valid = s[valid_mask]

        try:
            if method == "quantile":
                # equal frequency binning
                binned = pd.qcut(s_valid, q=n_bins, duplicates="drop")
            else:
                # equal width binning
                binned = pd.cut(s_valid, bins=n_bins, duplicates="drop")
        except ValueError:
            # e.g. all values are identical
            binned = pd.Series(["ALL"] * valid_mask.sum(), index=s_valid.index)

        bins[valid_mask] = binned.astype(str)
        bins[missing_mask] = "MISSING"
        return bins

    # ================== tree-based binning ==================
    elif method == "tree":
        if y is None:
            raise ValueError("y must be provided when method='tree'.")

        valid_mask = ~missing_mask
        if valid_mask.sum() == 0:
            bins[missing_mask] = "MISSING"
            return bins

        X = s[valid_mask].values.reshape(-1, 1)
        y_vec = np.array(y)[valid_mask]

        # ensure minimum samples per leaf
        min_samples_leaf = max(int(len(X) * min_leaf_frac), 1)

        clf = DecisionTreeClassifier(
            max_leaf_nodes=n_bins,
            min_samples_leaf=min_samples_leaf,
        )
        clf.fit(X, y_vec)
        leaf_ids = clf.apply(X)

        leaf_series = pd.Series(leaf_ids, index=s[valid_mask].index)
        bins[valid_mask] = leaf_series.map(lambda v: f"leaf_{int(v)}")
        bins[missing_mask] = "MISSING"
        return bins

    else:
        raise ValueError(f"Unsupported binning method: {method}")


tools = [bin_single_feature_tool]

agent = create_agent(
    model=model,
    tools=tools,
    context_schema=UserContext,
    system_prompt="You are a helpful assistant."
)


Message = {
    "messages": [
        {
          "role":    "user", 
          "content": "将特征 series 分箱，方法为 'quantile'，分箱数为 10"
        }
    ]
}

result = agent.invoke(Message, 
                      context = UserContext(user_id="user123"))

result['messages']


PydanticInvalidForJsonSchema: Cannot generate a JsonSchema for core_schema.IsInstanceSchema (<class 'pandas.core.series.Series'>)

For further information visit https://errors.pydantic.dev/2.12/u/invalid-for-json-schema

In [20]:
import pandas as pd
import numpy as np
from typing import List, Optional, Dict, Any
from sklearn.tree import DecisionTreeClassifier
from langchain.tools import tool

# ============== 1. Binning helper =====================
@tool
def bin_single_feature_tool(
    values: List[float],
    indices: Optional[List[Any]] = None,
    y_values: Optional[List[float]] = None,
    y_indices: Optional[List[Any]] = None,
    method: str = "quantile",   # 'quantile' | 'width' | 'tree'
    n_bins: int = 10,
    min_leaf_frac: float = 0.05,
) -> Dict[str, Any]:
    """
    Bin a single numeric feature according to the specified method.

    Parameters
    ----------
    values : List[float]
        List of raw feature values.
    indices : Optional[List[Any]]
        Optional list of indices corresponding to values.
    y_values : Optional[List[float]]
        Optional list of target labels (only needed when method='tree').
    y_indices : Optional[List[Any]]
        Optional list of indices corresponding to target values.
    method : str
        'quantile' (equal frequency), 'width' (equal width),
        or 'tree' (decision-tree-based optimal binning).
    n_bins : int
        Number of bins or max leaf nodes (for tree).
    min_leaf_frac : float
        Minimum fraction of samples per leaf for tree binning.

    Returns
    -------
    Dict[str, Any]
        Dictionary containing 'values' (binned values) and optional 'indices'.
        Missing values are grouped into 'MISSING'.
    """
    # 创建索引
    if indices is None:
        indices = list(range(len(values)))
    
    # 转换为pandas Series进行处理
    s = pd.Series(values, index=indices)
    s = pd.to_numeric(s, errors="coerce")
    bins = pd.Series(index=s.index, dtype="object")

    # handle missing values as a separate bin
    missing_mask = s.isna()

    # ================== quantile / width ==================
    if method in ("quantile", "width"):
        valid_mask = ~missing_mask
        if valid_mask.sum() == 0:
            # all values are missing
            bins[missing_mask] = "MISSING"
            return {"values": bins.tolist(), "indices": indices}

        s_valid = s[valid_mask]

        try:
            if method == "quantile":
                # equal frequency binning
                binned = pd.qcut(s_valid, q=n_bins, duplicates="drop")
            else:
                # equal width binning
                binned = pd.cut(s_valid, bins=n_bins, duplicates="drop")
        except ValueError:
            # e.g. all values are identical
            binned = pd.Series(["ALL"] * valid_mask.sum(), index=s_valid.index)

        bins[valid_mask] = binned.astype(str)
        bins[missing_mask] = "MISSING"
        return {"values": bins.tolist(), "indices": indices}

    # ================== tree-based binning ==================
    elif method == "tree":
        if y_values is None:
            raise ValueError("y_values must be provided when method='tree'.")

        # 创建目标变量的Series
        if y_indices is None:
            y_indices = indices
        y_series = pd.Series(y_values, index=y_indices)
        
        valid_mask = ~missing_mask
        if valid_mask.sum() == 0:
            bins[missing_mask] = "MISSING"
            return {"values": bins.tolist(), "indices": indices}

        X = s[valid_mask].values.reshape(-1, 1)
        # 确保只使用与X对应的y值
        y_vec = y_series.loc[s[valid_mask].index].values

        # ensure minimum samples per leaf
        min_samples_leaf = max(int(len(X) * min_leaf_frac), 1)

        clf = DecisionTreeClassifier(
            max_leaf_nodes=n_bins,
            min_samples_leaf=min_samples_leaf,
        )
        clf.fit(X, y_vec)
        leaf_ids = clf.apply(X)

        leaf_series = pd.Series(leaf_ids, index=s[valid_mask].index)
        bins[valid_mask] = leaf_series.map(lambda v: f"leaf_{int(v)}")
        bins[missing_mask] = "MISSING"
        return {"values": bins.tolist(), "indices": indices}

    else:
        raise ValueError(f"Unsupported binning method: {method}")


tools = [bin_single_feature_tool]

agent = create_agent(
    model=model,
    tools=tools,
    context_schema=UserContext,
    system_prompt="You are a helpful assistant."
)

Message = {
    "messages": [
        {
            "role": "user", 
            "content": "将特征 series 分箱，方法为 'quantile'，分箱数为 10"
        }
    ]
}

#result = agent.stream(Message, context = UserContext(user_id="user123"))

#result = agent.invoke(Message, context = UserContext(user_id="user123"))
#result['messages']
# agent_state = {
#      "Message": ["help me to bin the feature series using 'quantile' method with 10 bins."],
#      "findings": []
# }

# result = agent.invoke(agent_state,
#                       context=UserContext(user_id="user123"))

    
# print(result['messages'][-1].content)

In [22]:
for chunk in agent.stream(Message, stream_mode="messages"):
    print(chunk[0].content, end='')

我需要将特征 series 进行分箱处理，但您没有提供具体的数值数据。为了使用分箱工具，我需要知道：

1. **特征值列表**：请提供需要分箱的具体数值数据
2. **可选的索引**：如果有对应的索引信息也可以提供

例如，如果您有类似这样的数据：
- 特征值：[1.2, 3.4, 5.6, 7.8, 9.1, 2.3, 4.5, 6.7, 8.9, 0.1]
- 索引：[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

请提供您的具体数据，我将使用 'quantile' 方法将其分成 10 个等频分箱。